# ファイル操作

In [1]:
import java.io.*;
import java.nio.file.*;
import java.util.Arrays;

* 相対パスからパスを取得する関数

In [2]:
Path path(String relpath) {
	return Paths.get(System.getProperty("user.dir"),relpath);
}

* カレントディレクトリをホームに変更

In [ ]:
System.setProperty("user.dir", home);

## ファイル/フォルダ/シンボリックリンクの作成

* Emptyという空フォルダを作成

In [2]:
// 既にEmptyという名前のファイル/フォルダが存在していれば,エラーを投げる
Files.createDirectory(path("Empty"));

true

* Blankという空ファイルを作成

In [3]:
// 既にBlankという名前のファイル/フォルダが存在していれば,エラーを投げる
Files.createFile(path("Blank"));

true

* 一時ディレクトリの場合
```Java
Files.createTempDirectory("uniqueIdentifier"); // ディレクトリのパス
```

* フォルダEmptyの中にファイルBlankのシンボリックリンクSymlinkを作成

In [4]:
// 既にSymlinkという名前のファイル/フォルダが存在していれば,エラーを投げる
Files.createSymbolicLink(path("Empty/Symlink"),path("../Blank"));

true

- ハードリンクの場合
```Java
File.createLink(path("../Blank"),path("Empty/Link"))
```

## 書込み
* Untitled.mdというMarkdownファイルを作成して書込み

In [6]:
Files.write(
	path("Untitled.md"),
	Arrays.asList("# Header 1"),
	StandardOpenOption.CREATE
);

- オプションを `CREATE` から `CREATE_NEW` にすると,既にファイルが存在している場合はエラーになる
- `CREATE` の場合は,既に存在している場合は上書きされる

## 移動/名称変更

* フォルダEmptyをPackageに名称変更

In [7]:
Files.move(
	path("Empty"),
	path("Package"),
	StandardCopyOption.REPLACE_EXISTING
);

'Package'

* Packageフォルダ内のSymlinkファイルをAliasに名称変更

In [8]:
Files.move(
	path("Package/Symlink"),
	path("Package/Alias"),
	StandardCopyOption.REPLACE_EXISTING
);

'Package/Alias'

* Untitled.mdを移動して,名称変更

In [9]:
Files.move(
	path("Untitled.md"),
	path("Package/Headers.md"),
	StandardCopyOption.REPLACE_EXISTING
);

'Package/Headers.md'

## 追記
* Markdownファイルに追記

In [ ]:
Files.write(
	path("Package/Headers.md"),
	Arrays.asList("## Header 2\n### Header 3"),
	StandardOpenOption.APPEND
);

- `APPEND` の代わりに` WRITE` を使用すれば,先頭から上書きされる
- 但し, `WRITE` を使用する際には `TRUNCATE_EXISTING` も指定しないと単に上書きするだけで,前の内容が残る可能性がある。

## 読込み
* Markdownファイルを読込み

In [ ]:
String.join("\n",Files.readAllLines(path("Package/Headers.md")));

## 再帰的にフォルダ作成
* フォルダを一気に作成

In [ ]:
Files.createDirectories(path("Package/Module/Submodule/Item"));

- `Files.createDirectory` の代わりに `Files.createDirectories` を使うことで,作成するフォルダItemの上位フォルダModule,Submoduleが存在していなくても,同時に生成される

## ディレクトリの内容を表示

In [ ]:
File[] fl=new File(path("Package")).listFiles();
for (File f:fl) System.out.println(f.getName());

## ファイル/フォルダの複製
* Blankを複製

In [14]:
Files.copy(
	Paths.get("Blank"),
	path("Package/Blank"),
	StandardCopyOption.REPLACE_EXISTING
);

'Package/Module Copy'

- JDK標準ライブラリでディレクトリを再帰的にコピーする方法は存在しない  
	外部のライブラリを用意するか,再帰的にコピーするコードを自分で書かねばならない

## ファイル/フォルダの削除

* ファイルBlankを削除

In [15]:
Files.deleteIfExists(path("Blank"));

* シンボリックリンクAliasを削除

In [3]:
Files.deleteIfExists(path("Package/Alias"));

false

- File.deleteやFile.deleteIfExistsは空のディレクトリも削除できるが,空でないディレクトリを削除しようとするとエラーになる。  
	JDK標準ライブラリでディレクトリを再帰的に削除する方法は存在しない  
	外部のライブラリを用意するか,再帰的に削除するコードを自分で書かねばならない

## 権限を確認/変更

* 状態を確認する関数

In [ ]:
void check(String path) throws IOException {
	Path pt = path(path);
	System.out.println((Files.exists(pt)?"存在しています":"存在していません"));
	if (Files.isSymbolicLink(pt)) {
		System.out.println(
			"シンボリックリンクです"+os.linesep+
			"リンク先: "+Files.readSymbolicLink(pt)
		);
		if (Files.isRegularFile(pt)) System.out.println("リンク先はファイルです");
		if (Files.isDirectory(pt)) System.out.println("リンク先はフォルダです");
	}
	else {
		if (Files.isRegularFile(pt, LinkOption.NOFOLLOW_LINKS)) System.out.println("ファイルです");
		if (Files.isDirectory(pt, LinkOption.NOFOLLOW_LINKS)) System.out.println("フォルダです");
	}
	if (Files.isReadable(pt))  System.out.println("読込可能です");
	if (Files.isWritable(pt))  System.out.println("書込可能です");
	if (Files.isExecutable(pt)) System.out.println("実行可能です");
	if (Files.isHidden(pt)) System.out.println(" 不可視項目です");
}

In [ ]:
File fo=new File("Package/Headers.md");

* Markdownファイルの現在の状態を確認

In [19]:
check("Package/Headers.md")

存在しています
ファイルです
読込可能です
書込可能です


* 読込/書込権限を剥奪

In [20]:
fo.setReadable(false);
fo.setWritable(false);

* 状態を確認

In [21]:
check("Package/Headers.md")

存在しています
ファイルです


* 読込/実行権限を付加

In [22]:
fo.setReadable(true);
fo.setExecutable(true);

* 状態を確認

In [23]:
check("Package/Headers.md")

存在しています
ファイルです
読込可能です
実行可能です
